In [7]:
# %% [markdown]
# # Запуск проекта автодополнения текста из модулей с конфигурацией YAML

# %%
# Автоматическая перезагрузка модулей при изменениях кода
%load_ext autoreload
%autoreload 2

# %%
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

from data_utils import process_file_and_save, tokens_to_indices, pad_sequences_torch, compute_rouge
from next_token_dataset import TextDataset
from lstm_model import LSTMModel
from eval_lstm import train_epoch, eval_epoch
from transformers import pipeline, AutoTokenizer
import re
import yaml

# Загрузка конфигурации
with open('/home/assistant/text-autocomplete/config.yaml', 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

print("Текущая рабочая директория:", os.getcwd())

# %%
# Настройка устройства с учетом конфига
device_name = config['environment']['device']
device = torch.device(device_name if torch.cuda.is_available() and device_name == 'cuda' else 'cpu')
print(f"Используем устройство: {device}")

# %%
# Пути из конфига
paths = config['project']['paths']
input_file = paths['input_file']

# %%
# Обработка данных
samples = process_file_and_save(input_file)

# Создание словаря токенов
all_tokens = [token for sample in samples for token in sample['X']] + [token for sample in samples for token in sample['Y']]
vocab = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
vocab_size = len(vocab) + 1

# Индексация и паддинг
X_indices, Y_indices = tokens_to_indices(samples, vocab)
max_len = max(len(x) for x in X_indices)

X_pad = pad_sequences_torch(X_indices, max_len, torch)
Y_pad = pad_sequences_torch(Y_indices, max_len, torch)

# %%
# Разделение на train/val/test
split_cfg = config['dataset']['train_val_test_split']
X_train, X_temp, Y_train, Y_temp = train_test_split(X_pad, Y_pad, test_size=split_cfg['val_size'], random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=split_cfg['test_size_within_val'], random_state=42)

# Сохранение CSV выборок
os.makedirs(paths['data_dir'], exist_ok=True)
def tensor_to_str_list(tensor):
    return [' '.join(map(str, seq.tolist())) for seq in tensor]

pd.DataFrame({'X': tensor_to_str_list(X_train), 'Y': tensor_to_str_list(Y_train)}).to_csv(paths['train_data'], index=False, encoding=config['data_processing']['encoding'])
pd.DataFrame({'X': tensor_to_str_list(X_val), 'Y': tensor_to_str_list(Y_val)}).to_csv(paths['val_data'], index=False, encoding=config['data_processing']['encoding'])
pd.DataFrame({'X': tensor_to_str_list(X_test), 'Y': tensor_to_str_list(Y_test)}).to_csv(paths['test_data'], index=False, encoding=config['data_processing']['encoding'])

# %%
# DataLoader
batch_size = config['dataset']['batch_size']
train_dataset = TextDataset(X_train, Y_train)
val_dataset = TextDataset(X_val, Y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

# %%
# Инициализация модели, loss и оптимизатора с параметрами из конфига
lstm_cfg = config['model']['lstm']
embedding_dim = lstm_cfg['embedding_dim']
hidden_dim = lstm_cfg['hidden_dim']
padding_idx = lstm_cfg['padding_idx']

model = LSTMModel(vocab_size, embedding_dim, hidden_dim).to(device)

training_cfg = config['training']
criterion = torch.nn.CrossEntropyLoss(ignore_index=training_cfg['loss_function']['ignore_index'])
optimizer_cls = getattr(torch.optim, training_cfg['optimizer']['type'])
optimizer = optimizer_cls(model.parameters(), **training_cfg['optimizer']['params'])

# %%
# Обучение и оценка модели
epochs = training_cfg['epochs']
train_losses = []
val_losses = []

for epoch in range(epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss = eval_epoch(model, val_loader, criterion, device)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    if training_cfg['logging']['print_epoch_results']:
        print(f"Epoch {epoch+1}/{epochs} - train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")

# %%
# График потерь
plot_cfg = config['output']['plots']
plt.figure(figsize=plot_cfg['figsize'])
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
plt.xlabel(plot_cfg['xlabel'])
plt.ylabel(plot_cfg['ylabel'])
plt.title(plot_cfg['title'])
if plot_cfg['legend']:
    plt.legend()
plt.show()

# %%
# Сохранение модели
os.makedirs(os.path.dirname(paths['model_weights']), exist_ok=True)
torch.save(model.state_dict(), paths['model_weights'])
print(f"Модель сохранена по пути {paths['model_weights']}")

# %%
# Оценка distilgpt2 из отдельного блока с конфигом

generator_distilgpt2 = pipeline("text-generation", model=config['tokenizer']['distilgpt2_model_name'])
tokenizer_distilgpt2 = AutoTokenizer.from_pretrained(config['tokenizer']['distilgpt2_model_name'])

def split_text_for_completion(text):
    tokens = tokenizer_distilgpt2.tokenize(text)
    cut_off = (len(tokens) * 3) // 4
    input_text = tokenizer_distilgpt2.convert_tokens_to_string(tokens[:cut_off])
    target_text = tokenizer_distilgpt2.convert_tokens_to_string(tokens[cut_off:])
    return input_text, target_text

def clean_text_for_distilgpt2(text):
    text = text.lower()
    text = re.sub(config['data_processing']['clean_text_regex']['urls'], '', text)
    text = re.sub(config['data_processing']['clean_text_regex']['mentions'], '', text)
    # Сохраняем a-z, а-я для distilgpt2
    text = re.sub(r'[^\w\sа-яёa-z]', '', text, flags=re.UNICODE)
    text = re.sub(config['data_processing']['clean_text_regex']['whitespace'], ' ', text).strip()
    return text

texts = [clean_text_for_distilgpt2(t) for t in open(input_file, encoding=config['data_processing']['encoding']).readlines() if len(t.strip()) > 10]
_, val_texts = train_test_split(texts, test_size=split_cfg['val_size'], random_state=42)

rouge1_gpt2_scores, rouge2_gpt2_scores = [], []

print("Оцениваем модель distilgpt2...")

max_examples = config['evaluation']['validation_samples_limit']
print_limit = config['evaluation']['print_examples_limit']

for idx, text in enumerate(val_texts[:max_examples]):
    input_text, ref_text = split_text_for_completion(text)

    max_len_gpt2 = len(tokenizer_distilgpt2.encode(input_text + ref_text))
    gpt2_out = generator_distilgpt2(
        input_text,
        max_length=max_len_gpt2,
        do_sample=config['model']['distilgpt2']['use_sampling'],
        top_k=config['model']['distilgpt2']['top_k'],
        num_return_sequences=config['model']['distilgpt2']['num_return_sequences']
    )
    gpt2_pred = gpt2_out[0]['generated_text'][len(input_text):].strip()
    r1_gpt2, r2_gpt2 = compute_rouge(ref_text, gpt2_pred)
    rouge1_gpt2_scores.append(r1_gpt2)
    rouge2_gpt2_scores.append(r2_gpt2)

    if idx < print_limit:
        print(f"\nПример #{idx+1}")
        print("Вход:", input_text)
        print("Эталон:", ref_text)
        print(f"distilgpt2 предсказание: {gpt2_pred}")
        print(f"ROUGE-1: {r1_gpt2:.3f}, ROUGE-2: {r2_gpt2:.3f}")

print(f"\nСреднее ROUGE-1 distilgpt2: {np.mean(rouge1_gpt2_scores):.3f}")
print(f"Среднее ROUGE-2 distilgpt2: {np.mean(rouge2_gpt2_scores):.3f}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Текущая рабочая директория: /home/assistant/text-autocomplete/src
Используем устройство: cuda


KeyboardInterrupt: 